# Coupled multiscale problems: discretizations and preconditioning


## Nordic Graduate Coarse on Computational Mathematical Modeling
### Miroslav Kuchta (miroslav.kuchta@gmail.com)

The following notebook contains set of exercises accopanying the lecture. Prerequisites for executing the code are working installations of [FEniCS](https://fenicsproject.org/download/), [cbc.block](https://bitbucket.org/fenics-apps/cbc.block) and [fenics_ii](https://github.com/MiroK/fenics_ii). In addition to the materials listed in ''preconditioning'' notebook I have used the following in preparing the lecture

- [Finite Elements: Theory, Fast Solvers, and Applications in Solid Mechanics](https://books.google.no/books/about/Finite_Elements.html?id=DXin1jNR4ioC&redir_esc=y)
- [A Note on Preconditioning for Indefinite Linear Systems](http://epubs.siam.org/doi/abs/10.1137/S1064827599355153?journalCode=sjoce3)
- [A Preconditioned Iterative Method for Saddlepoint Problems](http://epubs.siam.org/doi/abs/10.1137/0613054)
- [The finite element method with Lagrangian multipliers](https://link.springer.com/article/10.1007/BF01436561)
- [On The Coupling of 1d and 3d Diffusion-Reaction Equations: Application to Tissue Perfusion Problems](http://www.worldscientific.com/doi/abs/10.1142/S0218202508003108)
- [Natural Preconditioning and Iterative Methods for Saddle Point Systems](http://epubs.siam.org/doi/abs/10.1137/130934921)

The lecture is divided into two parts; the first being dedicated to saddle point / coupled problems in general, the second part then discusses coupled multiscale problems.

In [ ]:
from dolfin import Expression
from sympy.printing import ccode
from sympy import symbols


def expr_body(expr, **kwargs):
    if not hasattr(expr, '__len__'):
        # Defined in terms of some coordinates
        xyz = set(symbols('x[0], x[1], x[2]'))
        xyz_used = xyz & expr.free_symbols
        assert xyz_used <= xyz
        # Expression params which need default values
        params = (expr.free_symbols - xyz_used) & set(kwargs.keys())
        # Body
        expr = ccode(expr).replace('M_PI', 'pi')
        # Default to zero
        kwargs.update(dict((str(p), 0.) for p in params))
        # Convert
        return expr
    # Vectors, Matrices as iterables of expressions
    else:
        return [expr_body(e, **kwargs) for e in expr]
    
    
def as_expression(expr, degree=4, **kwargs): 
    '''Turns sympy expressions to Dolfin expressions.'''
    return Expression(expr_body(expr), degree=degree, **kwargs)

## Saddle point systems

In the lecture you were told that $P_0-P_1$ element is inf-sup stable for certain formulation of mixed Poisson problem.
Let's verify that statement experimentally.

In [ ]:
from dolfin import *

def mixed_poisson(n, f):
    '''
    Solve 
    
        div(grad u) = -f in [0, 1]^2
                  u = 0  on boundary
              
    as
    
        sigma - grad(u) = 0    in [0, 1]^2 
        div(sigma)      = -f
                       u = 0 on boundary
                  
    by direct solver.
    '''
    mesh = UnitSquareMesh(n, n)
    
    V = VectorElement('Discontinuous Lagrange', mesh.ufl_cell(), 0)
    Q = FiniteElement('Lagrange', mesh.ufl_cell(), 1)
    W = FunctionSpace(mesh, MixedElement([V, Q]))
    
    sigma, u = TrialFunctions(W)
    tau, v = TestFunctions(W)
    
    a = inner(sigma, tau)*dx - inner(grad(u), tau)*dx - inner(grad(v), sigma)*dx
    L = -inner(f, v)*dx
    bc = DirichletBC(W.sub(1), Constant(0), 'on_boundary')
    
    wh = Function(W)
    solve(a == L, wh, bc)
    
    sigmah, uh = wh.split(deepcopy=True)
    
    return mesh.hmin(), sigmah, uh

In [ ]:
# Setup for Mixed Poisson, direct, to check stability by lookin at error convergence
import sympy as sp

x, y = sp.symbols('x[0] x[1]')

u = x*(1-x)*y*(1-y)*sp.sin(sp.pi*(x+y))
f = -(u.diff(x, 2) + u.diff(y, 2))
sigma = (u.diff(x, 1), u.diff(y, 1))

(u, f, sigma) = map(as_expression, (u, f, sigma))

hs, sigmas, us = [], [], []
for n in (2, 4, 8, 16, 32, 64):
    h, sigmah, uh = mixed_poisson(n, f)
    es, eu = errornorm(sigma, sigmah, 'L2'), errornorm(u, uh, 'H10')
    
    if hs:
        rates = ln(es/sigmas[-1])/ln(h/hs[-1])
        rateu = ln(eu/us[-1])/ln(h/hs[-1])
    else:
        rates, rateu = -1, -1
    print '|sigma-sigmah|_0 = %g(%.2f), |u-uh|_1 = %g(%.2f)' % (es, rates, eu, rateu)
    
    hs.append(h); sigmas.append(es); us.append(eu)

Prior to discussing a stable element pair we found a pair of spaces $V$, $Q$ for $\sigma$, $u$ such that the continuous problem is well-posed. Recall that this was $Q=L^2(\Omega)$, $V=H^1_0(\Omega)$. Below we make block-diagonal preconditioner based on Riesz maps. Blocks M, N are based respectively on the maps w.r.t to $L^2$ and $H^1_0$ inner products and algebraic multigrid is used to get the approxima inverses.

In [ ]:
from block import block_bc, block_assemble, block_mat, block_vec
from block.iterative import MinRes
from block.algebraic.petsc import AMG, Jacobi


def mixed_poisson_block(n, f):
    '''
    Solve 
    
        div(grad u) = -f in [0, 1]^2
                  u = 0  on boundary
              
    as
    
        sigma - grad(u) = 0    in [0, 1]^2 
        div(sigma)      = -f
                      u = 0 on boundary
                  
    by MinRes preconditioned with Riesz map preconditioner based on 
    (L2, (sigma, tau)*dx) and (H10, (grad(u), grad(v))*dx).
    '''
    mesh = UnitSquareMesh(n, n)
    
    V = VectorElement('Lagrange', mesh.ufl_cell(), 1)
    Q = FiniteElement('Lagrange', mesh.ufl_cell(), 2)
    W = [FunctionSpace(mesh, X) for X in (V, Q)]
    
    sigma, u = map(TrialFunction, W)
    tau, v = map(TestFunction, W)
    
    # Block matrix
    a00 = inner(sigma, tau)*dx
    a01 = -inner(grad(u), tau)*dx
    a10 = -inner(grad(v), sigma)*dx
    AA = block_assemble([[a00, a01], [a10,   0]])
    
    # Block vector
    L0 = inner(Constant((0, 0)), tau)*dx
    L1 = -inner(f, v)*dx
    bb = block_assemble([L0, L1])
    
    # True for symmetry
    bcsQ = [DirichletBC(W[1], Constant(0), 'on_boundary')]
    bcs = block_bc([[], bcsQ], True)
    bcs.apply(AA).apply(bb)
    
    # Inner products for Riesz map
    M = assemble(inner(sigma, tau)*dx)
    N, _ = assemble_system(inner(grad(u), grad(v))*dx, L1, bcsQ)
    # Now Riesz
    M = AMG(M)
    N = AMG(N)
    BB = block_mat([[M, 0], [0, N]])
    
    sigmah, uh = map(Function, W)
    
    # Setup and solve with Krylov
    x = block_vec([sigmah.vector(), uh.vector()])
    x.randomize()
    AAinv = MinRes(AA, precond=BB, tolerance=1e-10, show=2, initial_guess=x)
    x = AAinv*bb
    
    # Turn the coeff vectors to functions
    sigmah.vector()[:] = x[0]
    uh.vector()[:] = x[1]
    
    return mesh.hmin(), sigmah, uh, len(AAinv.residuals)-1

In [ ]:
# Setup for Mixed Poisson, iterative.
# Checking error convergence and boundedness of iteration in h
import sympy as sp

x, y = sp.symbols('x[0] x[1]')

u = x*(1-x)*y*(1-y)*sp.sin(sp.pi*(x+y))
f = -(u.diff(x, 2) + u.diff(y, 2))
sigma = (u.diff(x, 1), u.diff(y, 1))

(u, f, sigma) = map(as_expression, (u, f, sigma))

hs, sigmas, us = [], [], []
for n in (2, 4, 8, 16, 32, 64, 128):
    h, sigmah, uh, niters = mixed_poisson_block(n, f)
    es, eu = errornorm(sigma, sigmah, 'L2'), errornorm(u, uh, 'H10')
    
    if hs:
        rates = ln(es/sigmas[-1])/ln(h/hs[-1])
        rateu = ln(eu/us[-1])/ln(h/hs[-1])
    else:
        rates, rateu = -1, -1
    print '|sigma-sigmah|_0 = %g(%.2f), |u-uh|_1 = %g(%.2f), niters = %d' % (es, rates, eu, rateu, niters)
    
    hs.append(h); sigmas.append(es); us.append(eu)

**Exercise**
- Consider the M block and think about the structure/sparsity of the matrix. Can you use a cheaper approximation than AMG, e.g. Jacobi? How expensive is the exact inverse?
- Can you base N block on the $H^1$ inner product?
- Is (`FiniteElement('Lagrange', triangle, 1)`, `FiniteElement('Lagrange', triangle, 2)`) a _stable_ element?
- How do the MinRes iterations behave if this element is used in the original `mixed_poisson_block`?
- Keep the $P_2$ element for $Q$ and find an element for $V$ such that the pair is _stable_.

_stable_ = you have numerical evidence for it

## Coupled multiscale problems
### Fractional spaces

_One cannot survive without the Laplacian. In mathematics, that is. In life, that depends on your definition of the Laplacian._, J. L. Vazquez, 2007

We considered an eigenvalue problem $-u''=\lambda u$ on $(0, 1)$ with zero Dirichlet boundary conditions concluding that the eigenfunctions $\phi_k=\sqrt(2)\sin k\pi x$ form an $L^2$ orthonormal basis of the space $L^2(\Omega)$. For $s\in\left[-1, 1\right]$ we then defined $(-\Delta)^s$ such that $(-\Delta)^s u$ = $\sum_k \lambda_k^s c_k \phi_k$ where $u=c_k \phi_k$ and $\lambda_k=(k\pi)^2$ are the eigenvalues of the Laplacian. Finally we defined the space $H^s$ and related the property $u\in H^s$ to decay rates of the Fourier coefficients.

Consider et $u = \begin{cases}
2x & 0 \leq x \leq 1/2 \\
2x-2 & 1/2 < x \leq 1
\end{cases}$. This is not a H^1_0((0, 1)) functions. Let's about its power spectrum.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

x = np.linspace(0, 1, 1000)
f = np.zeros_like(x)

A = sqrt(2)
k_max = 10000
ks = np.arange(1, k_max+1)
spectrum = np.zeros_like(ks, dtype=float)
for k in ks:
    ck = -2*A*cos(k*pi/2)/(k*pi)

    spectrum[k-1] = abs(ck)
    f += ck*A*np.sin(k*pi*x)

mask = np.where(spectrum > 1E-13)[0]
plt.loglog(ks[mask], spectrum[mask], 'o', linestyle='none')
plt.loglog(ks, 1./ks**2)

**Exercise**
- Let $u = \lvert 1/2-x \rvert$. Is this a $H^1_0((0, 1))$ function? What is the decay rate of its Fourier coefficients?

### Discrete fractional norms

We shall define the discrete approximation of $H^s$ norm by mirroring the continuous definition, i.e. the norms will be based (generalized) eigenvalue problem stemming from an eigenvalue problem $-u''=\lambda u$ on $(0, 1)$ with zero Dirichlet boundary conditions. 

In [ ]:
from scipy.linalg import eigh


def hs_norm(mesh, s, f):
    '''
    H^s norm on ((0, 1)) based on Laplace eigenvalue problem on the
    domain with zero Dirichlet bcs.
    '''
    assert mesh.geometry().dim() == 1
    assert mesh.num_vertices() < 5000
    
    V = FunctionSpace(mesh, 'CG', 1)
    bc = DirichletBC(V, Constant(0), 'on_boundary')
    u = TrialFunction(V)
    v = TestFunction(V)
    
    f_vec = interpolate(f, V).vector().array()
    # Setup generalized eigenvalue problem for discrete norm
    a = inner(grad(u), grad(v))*dx
    m = inner(u, v)*dx
    L = inner(f, v)*dx
    
    A, _ = assemble_system(a, L, bc)
    M, _ = assemble_system(m, L, bc)
    A, M = A.array(), M.array()

    lmbda, U = eigh(A, M)
    V = M.dot(U)
    Lambda = np.diag(lmbda**s)
    
    # Matrix representation of the norm
    H = V.dot(Lambda.dot(V.T))
    
    return sqrt(np.sum(f_vec*H.dot(f_vec)))

Let's verify that our new defintion works for $H^1_0$ and $L^2$ norms.

In [ ]:
for i in range(3, 11):
    mesh = UnitIntervalMesh(2**i)
    dX = Measure('dx', domain=mesh)
    
    V = FunctionSpace(mesh, 'CG', 2)
    f = interpolate(Expression('x[0]*(1-x[0])', degree=2), V)
    
    L2 = sqrt(assemble(inner(f, f)*dX))
    H10 = sqrt(assemble(inner(f.dx(0), f.dx(0))*dX))
    
    L2_ = hs_norm(mesh, s=0., f=f)
    H10_ = hs_norm(mesh, s=1., f=f)
    
    print abs(L2 - L2_), abs(H10 - H10_)

**Exercise**
- For a function of you choice compute its _continuous_ $H^{1/2}$ norm. How fast does the _discrete_ norm approximates the number? 

### Babuska's problem

In [ ]:
# PATH to fenics_ii if not there already
import sys
sys.path.append('/home/miro3/Documents/Programming/fenics_ii/py')


from dolfin import *
from trace_tools.trace_assembler import trace_assemble
from trace_tools.norms import inv_interpolation_norm_eig
from trace_tools.embedded_mesh import EmbeddedMesh
from block import block_mat, block_vec
from block.iterative import MinRes
from block.algebraic.petsc import AMG


def babuska(n, f, g, gamma):
    '''
    Minimize 0.5*inner(grad(u), grad(u))*dx+0.5*inner(u, u)*dx-inner(f, v) over 
    H^1(Omega) with Neumann bcs subject to Tu = g on Gamma.
    
    Usinf Riesz map peconditioner based on H^1 and H^{-0.5} spaces.
    '''
    n *= 4
    omega_mesh = UnitSquareMesh(n, n)
    facet_f = FacetFunction('size_t', omega_mesh, 0)
    gamma.mark(facet_f, 1)

    gamma_mesh = EmbeddedMesh(omega_mesh, facet_f, 1)

    # Space of u and the Lagrange multiplier
    V = FunctionSpace(omega_mesh, 'CG', 1)
    Q = FunctionSpace(gamma_mesh.mesh, 'CG', 1)

    u, p = TrialFunction(V), TrialFunction(Q)
    v, q = TestFunction(V), TestFunction(Q)

    dxGamma = Measure('dx', domain=gamma_mesh.mesh)

    a00 = inner(grad(u), grad(v))*dx + inner(u, v)*dx
    a01 = inner(p, v)*dxGamma
    a10 = inner(u, q)*dxGamma
    L0 = inner(f, v)*dx
    L1 = inner(g, q)*dxGamma

    # Blocks
    A00 = assemble(a00)
    A01 = trace_assemble(a01, gamma_mesh)
    A10 = trace_assemble(a10, gamma_mesh)
    b0 = assemble(L0)
    b1 = assemble(L1)
    # System
    AA = block_mat([[A00, A01], [A10, 0]])
    bb = block_vec([b0, b1])

    # Preconditioner blocks
    P00 = AMG(A00)
    # Trace of H^1 is H^{1/2} and the dual is H^{-1/2}
    m = inner(p, q)*dxGamma
    a = inner(grad(p), grad(q))*dxGamma + m
    P11 = inv_interpolation_norm_eig(a, s=-0.5, m=m)
    # P11 = 1.
    
    # The preconditioner
    BB = block_mat([[P00, 0], [0, P11]])

    AAinv = MinRes(AA, precond=BB, tolerance=1e-10, maxiter=300, show=0)
    # Compute solution
    U, P = AAinv * bb
    uh, ph = Function(V, U), Function(Q, P)
    
    niters = len(AAinv.residuals) - 1
    return (V.dim(), Q.dim(), niters)

Let's see the preconditioner in action.

In [ ]:
gamma = CompiledSubDomain('near(x[0], 0)')

f = Expression('sin(pi*(x[0]-x[1]))', degree=3)
g = Expression('cos(pi*(x[0]+x[1]))', degree=3)


for n in [2, 4, 8, 16, 32]:
    print 'dim(V) = %d dim(Q) = %d niters = %d' % babuska(n, f, g, gamma)

**Exercise**
- Based on characterization of trace as a bounded operator from $H^1(\Omega)$ to $L^2(\partial\Omega)$ you might want to consider the Lagrange multiplier ($p$) in the space $L^2(\partial\Omega)$ and base the preconditioner on the $L^2$ inner product. This way you avoid eigenvalue computations. Implement the idea. Do you get a good iterative solver this way?